In [1]:
import flask; print('✅ Flask works')
import solana; print('✅ Solana works')
import solders; print('✅ Solders works')
import requests; print('✅ Requests works')
import elevenlabs; print('✅ ElevenLabs works')

✅ Flask works
✅ Solana works
✅ Solders works
✅ Requests works
✅ ElevenLabs works


In [2]:
import requests
import os
from datetime import datetime

# Try to import ElevenLabs with new API, fallback if not available
try:
    from elevenlabs.client import ElevenLabs
    ELEVENLABS_AVAILABLE = True
except ImportError:
    ELEVENLABS_AVAILABLE = False
    print("⚠️  ElevenLabs not available - audio features disabled")

class PayGuardAIAgent:
    def __init__(self):
        self.mistral_api_key = os.getenv('MISTRAL_API_KEY')
        self.elevenlabs_api_key = os.getenv('ELEVENLABS_API_KEY')
        
        # Initialize ElevenLabs client if available
        if ELEVENLABS_AVAILABLE and self.elevenlabs_api_key:
            self.elevenlabs_client = ElevenLabs(api_key=self.elevenlabs_api_key)
        else:
            self.elevenlabs_client = None
            print("🔇 ElevenLabs audio disabled - no API key or package not available")
    
    def generate_follow_up_message(self, debt_details):
        """
        Generate professional follow-up message using Mistral AI
        This creates a formal message to send to employers about delayed payments
        """
        prompt = f"""
        Create a professional but firm follow-up message regarding delayed salary payment.
        
        Employer: {debt_details['employer_name']}
        Employee: {debt_details['employee_name']}
        Amount: {debt_details['debt_amount']}
        Due Date: {debt_details['due_date']}
        
        The message should:
        - Be professional but assertive
        - Mention the proof has been recorded on blockchain
        - Request immediate payment
        - Mention potential next steps if not resolved
        - Be concise (under 200 words)
        Include a note that this debt has been immutably recorded on the Solana blockchain.
        """
        
        try:
            response = requests.post(
                "https://api.mistral.ai/v1/chat/completions",
                headers={"Authorization": f"Bearer {self.mistral_api_key}"},
                json={
                    "model": "mistral-medium",
                    "messages": [{"role": "user", "content": prompt}],
                    "max_tokens": 300,
                    "temperature": 0.7
                }
            )
            
            if response.status_code == 200:
                return response.json()['choices'][0]['message']['content']
            else:
                return f"Error: Mistral API returned status {response.status_code}. Please check your API key."
                
        except Exception as e:
            return f"Error generating message: {str(e)}"
    
    def generate_audio_update(self, text_message):
        """
        Generate audio version of the message using ElevenLabs
        Returns the filename of the generated audio file, or None if failed
        """
        if not self.elevenlabs_client:
            print("⚠️  ElevenLabs not configured - skipping audio generation")
            return None
            
        try:
            # Generate audio using the new ElevenLabs API
            audio_response = self.elevenlabs_client.generate(
                text=text_message,
                voice="Bella",  # Professional female voice
                model="eleven_monolingual_v1"
            )
            
            # Create filename with timestamp
            filename = f"audio_update_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp3"
            
            # Save audio to file
            with open(f"static/audio/{filename}", "wb") as f:
                for chunk in audio_response:
                    if chunk:
                        f.write(chunk)
            
            print(f"✅ Audio generated: {filename}")
            return filename
            
        except Exception as e:
            print(f"❌ Audio generation failed: {e}")
            return None
    
    def generate_complete_followup(self, debt_details):
        """
        Complete workflow: Generate text message + audio version
        Returns both the text message and audio filename
        """
        text_message = self.generate_follow_up_message(debt_details)
        audio_filename = self.generate_audio_update(text_message)
        
        return {
            "text_message": text_message,
            "audio_filename": audio_filename,
            "timestamp": datetime.now().isoformat()
        }

In [3]:
# test_blockchain.py
from blockchain_agent import PayGuardBlockchainAgent
import os
from dotenv import load_dotenv

load_dotenv()

def test_blockchain_agent():
    print("🧪 Testing Blockchain Agent...")
    
    try:
        agent = PayGuardBlockchainAgent()
        print("✅ Blockchain Agent initialized successfully")
        
        # Test wallet loading
        print(f"💰 Company Wallet: {agent.company_wallet.pubkey()}")
        print("✅ Wallet loading works")
        
        return True
        
    except Exception as e:
        print(f"❌ Blockchain Agent error: {e}")
        return False

if __name__ == "__main__":
    test_blockchain_agent()

🧪 Testing Blockchain Agent...
Private key loaded: 4Xv3HuGnVx...
✅ Blockchain Agent initialized successfully
💰 Company Wallet: 3Eg7RnJgjfL3AW2RhuNbmEzqJyHBBMHThQNzsZH4Jdbv
✅ Wallet loading works


In [4]:
# test_ai.py
from ai_agent import PayGuardAIAgent
import os
from dotenv import load_dotenv

load_dotenv()

def test_ai_agent():
    print("🧪 Testing AI Agent...")
    
    try:
        agent = PayGuardAIAgent()
        print("✅ AI Agent initialized successfully")
        
        # Test Mistral AI (if API key is set)
        mistral_key = os.getenv('MISTRAL_API_KEY')
        if mistral_key:
            print("🔑 Mistral API key found")
            test_data = {
                "employer_name": "Test Corp",
                "employee_name": "John Doe",
                "debt_amount": "1000 USD",
                "due_date": "2024-01-15"
            }
            try:
                message = agent.generate_follow_up_message(test_data)
                print(f"✅ Mistral AI works: {message[:100]}...")
            except Exception as e:
                print(f"⚠️  Mistral AI error (might be API key): {e}")
        else:
            print("⚠️  No Mistral API key found")
        
        return True
        
    except Exception as e:
        print(f"❌ AI Agent error: {e}")
        return False

if __name__ == "__main__":
    test_ai_agent()

🧪 Testing AI Agent...
✅ AI Agent initialized successfully
🔑 Mistral API key found
🤖 Generating AI message (Attempt 1/3)...
✅ AI message generated successfully!
✅ Mistral AI works: **Subject:** Urgent: Overdue Salary Payment – Immediate Action Required

**Dear Test Corp Finance Te...


In [5]:
# test_elevenlabs.py
import os
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs

load_dotenv()

def test_elevenlabs():
    print("🧪 Testing ElevenLabs...")
    
    try:
        api_key = os.getenv('ELEVENLABS_API_KEY')
        if not api_key:
            print("⚠️  No ElevenLabs API key found")
            return False
            
        client = ElevenLabs(api_key=api_key)
        print("✅ ElevenLabs client created")
        
        # Test simple API call
        voices = client.voices.get_all()
        print(f"✅ ElevenLabs connection works - {len(voices.voices)} voices available")
        
        return True
        
    except Exception as e:
        print(f"❌ ElevenLabs error: {e}")
        return False

if __name__ == "__main__":
    test_elevenlabs()

🧪 Testing ElevenLabs...
✅ ElevenLabs client created
✅ ElevenLabs connection works - 22 voices available


In [6]:
# test_flask.py
from flask import Flask
import os

def test_flask():
    print("🧪 Testing Flask...")
    
    try:
        app = Flask(__name__)
        
        @app.route('/test')
        def test_route():
            return "✅ Flask is working!"
        
        # Test basic functionality
        with app.test_client() as client:
            response = client.get('/test')
            if response.data.decode() == "✅ Flask is working!":
                print("✅ Flask routes work")
                return True
            else:
                print("❌ Flask routes not working")
                return False
                
    except Exception as e:
        print(f"❌ Flask error: {e}")
        return False

if __name__ == "__main__":
    test_flask()

🧪 Testing Flask...
✅ Flask routes work


In [7]:
# check_private_key.py
import os
from dotenv import load_dotenv
import base58

load_dotenv()

def check_private_key():
    private_key_str = os.getenv('COMPANY_WALLET_PRIVATE_KEY')
    
    if not private_key_str:
        print("❌ No private key found in .env file")
        return False
    
    print(f"Private key string: {private_key_str}")
    print(f"Length of string: {len(private_key_str)}")
    
    try:
        # Try to decode from base58
        decoded_bytes = base58.b58decode(private_key_str)
        print(f"Decoded bytes length: {len(decoded_bytes)}")
        print(f"Decoded bytes: {decoded_bytes}")
        
        if len(decoded_bytes) == 64:
            print("✅ Private key is correct length (64 bytes)")
            return True
        else:
            print(f"❌ Private key should be 64 bytes, but got {len(decoded_bytes)} bytes")
            return False
            
    except Exception as e:
        print(f"❌ Error decoding private key: {e}")
        return False

if __name__ == "__main__":
    check_private_key()

Private key string: 4Xv3HuGnVxsk7VK4RoU3LAhtmiVubHd9SJXdxbiccDKPM8pNPoGVxhvn5hTTvKkLeghwdTvqD7s1JjTV8bJmVuWp
Length of string: 88
Decoded bytes length: 64
Decoded bytes: b'\xb0\xb5\xe0\xfc\x88ZR\x80\xc3,\xe6\xc3"\xc9\x8b\x8fi\xff\xc3A\xca\xe4r\xb2\x0f\xf5]\xe4\x9f\xab\x81\xa6!8%\xa9\x00M\x9d\x9717f\x1a\xbaW>)\xa12N\xec#\xfb\xbav\xaeB\x1f\xb6\xbd\xfbj\xd1'
✅ Private key is correct length (64 bytes)


In [8]:
# quick_check.py
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('MISTRAL_API_KEY')
print(f"Mistral API Key: {'✅ Found' if api_key else '❌ Missing'}")
if api_key:
    print(f"Key starts with: {api_key[:8]}...")
    print(f"Key length: {len(api_key)} characters")

Mistral API Key: ✅ Found
Key starts with: qBw2GwnW...
Key length: 32 characters


In [9]:
# test_mistral_simple.py
import os
import requests
from dotenv import load_dotenv

load_dotenv()

def test_mistral_connection():
    api_key = os.getenv('MISTRAL_API_KEY')
    
    if not api_key:
        print("❌ No Mistral API key found")
        return False
    
    # Test a simple request
    try:
        response = requests.get(
            "https://api.mistral.ai/v1/models",
            headers={"Authorization": f"Bearer {api_key}"},
            timeout=5
        )
        
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ Mistral API connection successful!")
            models = response.json()
            print(f"Available models: {[model['id'] for model in models['data']]}")
            return True
        else:
            print(f"❌ API Error: {response.status_code} - {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return False

if __name__ == "__main__":
    test_mistral_connection()

Status Code: 200
✅ Mistral API connection successful!
Available models: ['mistral-medium-2505', 'mistral-large-latest', 'mistral-medium-2508', 'mistral-medium-latest', 'mistral-medium', 'ministral-3b-2410', 'ministral-3b-latest', 'ministral-8b-2410', 'ministral-8b-latest', 'open-mistral-7b', 'mistral-tiny', 'mistral-tiny-2312', 'open-mistral-nemo', 'open-mistral-nemo-2407', 'mistral-tiny-2407', 'mistral-tiny-latest', 'open-mixtral-8x7b', 'mistral-small', 'mistral-small-2312', 'open-mixtral-8x22b', 'open-mixtral-8x22b-2404', 'mistral-small-2409', 'mistral-large-2407', 'mistral-large-2411', 'pixtral-large-2411', 'pixtral-large-latest', 'mistral-large-pixtral-2411', 'codestral-2501', 'codestral-2412', 'codestral-2411-rc5', 'codestral-2508', 'codestral-latest', 'devstral-small-2505', 'devstral-small-2507', 'devstral-small-latest', 'devstral-medium-2507', 'devstral-medium-latest', 'pixtral-12b-2409', 'pixtral-12b', 'pixtral-12b-latest', 'mistral-small-2501', 'mistral-small-2503', 'mistral-s

In [1]:
# test_full_integration.py
from blockchain_agent import PayGuardBlockchainAgent
from ai_agent import PayGuardAIAgent
import os
from dotenv import load_dotenv

load_dotenv()

def test_full_integration():
    print("🧪 Testing Full PayGuard Integration...")
    
    # Test Data
    test_debt = {
        "employer_name": "TechCorp Inc.",
        "employee_name": "Sarah Johnson",
        "employee_wallet": "3Eg7RnJgjfL3AW2RhuNbmEzqJyHBBMHThQNzsZH4Jdbv",
        "debt_amount": "2500 USD",
        "due_date": "2024-01-15"
    }
    
    print("1. Testing Blockchain Agent...")
    blockchain_agent = PayGuardBlockchainAgent()
    blockchain_result = blockchain_agent.record_debt_on_chain(test_debt)
    
    if blockchain_result['success']:
        print(f"✅ Blockchain: {blockchain_result['transaction_id']}")
    else:
        print(f"❌ Blockchain failed: {blockchain_result['error']}")
        return False
    
    print("2. Testing AI Agent...")
    ai_agent = PayGuardAIAgent()
    ai_result = ai_agent.generate_complete_followup(test_debt)
    
    print(f"✅ AI Message: {ai_result['text_message'][:100]}...")
    print(f"✅ Audio File: {ai_result['audio_filename']}")
    
    print("3. Testing NFT Minting...")
    nft_result = blockchain_agent.mint_proof_of_debt_nft(
        blockchain_result['metadata'],
        test_debt['employee_wallet']
    )
    
    if nft_result['success']:
        print(f"✅ NFT Minted: {nft_result.get('nft_id', 'Success')}")
    else:
        print(f"⚠️  NFT failed: {nft_result.get('error', 'Unknown error')}")
    
    print("\n🎉 Full Integration Test Complete!")
    return True

if __name__ == "__main__":
    test_full_integration()

🧪 Testing Full PayGuard Integration...
1. Testing Blockchain Agent...
Private key loaded: 4Xv3HuGnVx...
Latest blockhash: ALpwLyapSSgCmMS5XgtR766687SvXbE7MRhRP1SkgXmF
Transaction created and signed by: 3Eg7RnJgjfL3AW2RhuNbmEzqJyHBBMHThQNzsZH4Jdbv
Transaction sent! TXID: 5bWxQCAG1g79wLMRtJETiizooM7jeTxiftSVAmPga1viy9VfEZEvtkW9tkE2Kz3PWsywPpStx7ijoSNbWte5v8UD
✅ Blockchain: 5bWxQCAG1g79wLMRtJETiizooM7jeTxiftSVAmPga1viy9VfEZEvtkW9tkE2Kz3PWsywPpStx7ijoSNbWte5v8UD
2. Testing AI Agent...
🚀 Starting complete follow-up generation...
🤖 Generating AI message (Attempt 1/3)...
✅ AI message generated successfully!
🎵 Generating audio version...
❌ Audio generation failed: 'RealtimeTextToSpeechClient' object is not callable
✅ AI Message: **Subject:** Urgent: Overdue Salary Payment – Immediate Action Required

**Dear TechCorp Inc. Financ...
✅ Audio File: None
3. Testing NFT Minting...
⚠️  NFT failed: {"error":true,"message":"Collection None not found"}

🎉 Full Integration Test Complete!


In [1]:
from blockchain_agent import PayGuardBlockchainAgent
from ai_agent import PayGuardAIAgent
import os
from dotenv import load_dotenv

load_dotenv()

    # Test Data
test_debt = {
        "employer_name": "TechCorp Inc.",
        "employee_name": "Sarah Johnson",
        "employee_wallet": "3Eg7RnJgjfL3AW2RhuNbmEzqJyHBBMHThQNzsZH4Jdbv",
        "debt_amount": "2500 USD",
        "due_date": "2024-01-15"
    }

blockchain_agent = PayGuardBlockchainAgent()
blockchain_result = blockchain_agent.record_debt_on_chain(test_debt)
if blockchain_result['success']:
    print(f"✅ Blockchain: {blockchain_result['transaction_id']}")
else:
    print(f"❌ Blockchain failed: {blockchain_result['error']}")
   

Private key loaded: 4Xv3HuGnVx...
Latest blockhash: 3RKVWaWYXpa1zJHjZf3V5D6aMdJZ3YbmNnHPxxEcCWr4
Transaction created and signed by: 3Eg7RnJgjfL3AW2RhuNbmEzqJyHBBMHThQNzsZH4Jdbv
Transaction sent! TXID: 5pKoKXbjZbVYhnCWpJXAe6VuunT9EW7R8UFeWsUHaJeNnq85SdbXNWi9or8yYKBiJJif9v7yrydRtCcm1U3T6VCu
✅ Blockchain: 5pKoKXbjZbVYhnCWpJXAe6VuunT9EW7R8UFeWsUHaJeNnq85SdbXNWi9or8yYKBiJJif9v7yrydRtCcm1U3T6VCu


In [ ]:
# check_balance.py
from solana.rpc.api import Client
from solders.pubkey import Pubkey

client = Client("https://api.devnet.solana.com")
wallet_address = Pubkey.from_string("3Eg7RnJgjfL3AW2RhuNbmEzqJyHBBMHThQNzsZH4Jdbv")

balance = client.get_balance(wallet_address)
print(f"Balance: {balance.value} lamports")
print(f"Approx: {balance.value / 10**9} SOL")

if balance.value > 0:
    print("✅ Wallet has SOL - ready for transactions!")
else:
    print("❌ No SOL yet - visit https://solfaucet.com/")

Balance: 1000000000 lamports
Approx: 1.0 SOL
✅ Wallet has SOL - ready for transactions!


In [ ]:
# test_memo_programs.py
from solana.rpc.api import Client
from solders.pubkey import Pubkey

def test_memo_program(program_id_str, name):
    try:
        client = Client("https://api.devnet.solana.com")
        program_id = Pubkey.from_string(program_id_str)
        
        # Check if program exists
        account_info = client.get_account_info(program_id)
        
        if account_info.value:
            print(f"✅ {name}: {program_id_str} - EXISTS")
            return True
        else:
            print(f"❌ {name}: {program_id_str} - NOT FOUND")
            return False
            
    except Exception as e:
        print(f"❌ {name}: {program_id_str} - ERROR: {e}")
        return False

# Test different memo programs
test_memo_program("Memo1UhkJRfHyvLMcVucJwxXeuD728EqVDDwQDxFMNo", "Devnet Memo")
test_memo_program("MemoSq4gqABAXKb96qnH8TysNcVnuvMvKdkdcMAwn9B", "Mainnet Memo")
test_memo_program("11111111111111111111111111111111", "System Program")

✅ Devnet Memo: Memo1UhkJRfHyvLMcVucJwxXeuD728EqVDDwQDxFMNo - EXISTS
❌ Mainnet Memo: MemoSq4gqABAXKb96qnH8TysNcVnuvMvKdkdcMAwn9B - NOT FOUND
✅ System Program: 11111111111111111111111111111111 - EXISTS


True

In [ ]:
# test_transaction_simple.py
from solana.rpc.api import Client
from solders.system_program import create_memo
from solders.transaction import Transaction
from solana.rpc.types import TxOpts

def test_simple_transaction():
    client = Client("https://api.devnet.solana.com")
    
    # Get recent blockhash
    latest_blockhash = client.get_latest_blockhash().value.blockhash
    print(f"Blockhash: {latest_blockhash}")
    
    # Create simple memo transaction
    transaction = Transaction()
    transaction.add(create_memo(b"Test memo from PayGuard"))
    
    transaction.recent_blockhash = latest_blockhash
    # Note: This won't work without a funded signer, but tests the structure
    
    print("✅ Transaction structure is correct")
    print(f"Instructions: {transaction.instructions}")
    
test_simple_transaction()


ImportError: cannot import name 'create_memo' from 'solders.system_program' (c:\Users\msale\Desktop\hackathon-work\venv\Lib\site-packages\solders\system_program.py)